In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import scale

## Load Data

In [2]:
df_movies = pd.read_csv('./movies_metadata_clean.csv', encoding='ISO-8859-1')
df_users = pd.read_csv('./movies_ratings_small.csv', encoding='ISO-8859-1')

In [3]:
# Replace spaces and remove characters

df_movies['clean_description'] = df_movies['original_title']
df_movies['clean_description'] = df_movies['clean_description'].str.replace(' ','_')
df_movies['clean_description'].str.replace('\W','')

df_users['clean_description'] = df_users['userId']
df_users['clean_description'] = df_users['clean_description'].replace(' ','_')
df_users['clean_description'].replace('\W','')

0                          Toy_Story
1                            Jumanji
2                   Grumpier_Old_Men
3                  Waiting_to_Exhale
4        Father_of_the_Bride_Part_II
                    ...             
45458                  ÃÂâÃ_ÃÃÅÃÃÃÃÂ
45459            Siglo_ng_Pagluluwal
45460                       Betrayal
45461            Satana_likuyushchiy
45462                       Queerama
Name: clean_description, Length: 45463, dtype: object

0           1
1           1
2           1
3           1
4           1
         ... 
99999     671
100000    671
100001    671
100002    671
100003    671
Name: clean_description, Length: 100004, dtype: int64

In [4]:
# Drop missing values
df_movies.dropna(inplace=True)
# df_users.dropna(inplace=True)

In [5]:
# Convert to list format

movie_list = df_users.groupby('rating').clean_description.apply(list)

## Build Model

Association Rule Model

In [6]:
# Encode data as transaction matrix

from mlxtend.preprocessing import TransactionEncoder

te_model = TransactionEncoder()
te_model_arr = te_model.fit(movie_list).transform(movie_list)
df_users = pd.DataFrame(te_model_arr, columns=te_model.columns_)
df_users.head()

,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
0,False,False,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,True,False,True,False,True,True,False,False,False,...,True,False,False,True,True,False,True,False,True,True
2,False,False,False,False,True,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
3,True,True,True,True,False,True,True,True,True,True,...,True,False,True,True,True,True,False,True,True,True
4,True,False,True,False,True,False,False,True,False,False,...,False,False,True,False,False,False,False,False,False,True


## Note
I am not sure why this code does not work. Tested with 36GB of RAM. The apriori function call used up all of the available RAM and Swap and locked up the computer.

In [7]:
# Determine the items and itemsets with at least 15% support

from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(df_users, min_support=0.15, use_colnames=True)
frequent_itemsets

KeyboardInterrupt: 

In [8]:
# Evaluate metrics and filter items/itemsets that have at least 70% confidence

from mlxtend.frequent_patterns import association_rules

a_rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.7)
a_rules.sort_values(by=['confidence'],ascending=False, inplace=True)
a_rules.head()

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set